About the Dataset:

1.id: unique id for a news article

2.title: the title of a news article

3.author: author of the news article

4.text: the text of the article; could be incomplete


5.label: a label that marks whether the news article is real or fake:

    1: Fake news
    0: real News


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import pickle
#
import re
from nltk.corpus import stopwords # to remove words which add no value to the data
from nltk.stem.porter import PorterStemmer
#
from sklearn.feature_extraction.text import TfidfVectorizer # text to feature vectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.metrics import accuracy_score,classification_report
#
import tensorflow as tf
from tensorflow.keras.models import Sequential # model
from tensorflow.keras.layers import Dense # type of layer
from tensorflow.keras.activations import linear, relu, sigmoid,softmax #activation function
from tensorflow.keras import regularizers

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:

#printing the stopwatch in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/FakeNewsNet.csv')

In [5]:
news_dataset.shape

(23196, 5)

In [6]:
# print the first 5 rows of the dataframe
news_dataset.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [7]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

title              0
news_url         330
source_domain    330
tweet_num          0
real               0
dtype: int64

In [10]:
# replacing the null values with empty string
news_dataset = news_dataset.dropna()

In [11]:
# merging the author name and news title
news_dataset['content'] = news_dataset['source_domain']+' '+news_dataset['title']

In [12]:
print(news_dataset['content'])

0        toofab.com Kandi Burruss Explodes Over Rape Ac...
1        www.today.com People's Choice Awards 2018: The...
2        www.etonline.com Sophia Bush Sends Sweet Birth...
3        www.dailymail.co.uk Colombian singer Maluma sp...
4        www.zerchoo.com Gossip Girl 10 Years Later: Ho...
                               ...                        
23191    www.express.co.uk Pippa Middleton wedding: In ...
23192    hollywoodlife.com Zayn Malik & Gigi Hadid’s Sh...
23193    www.justjared.com Jessica Chastain Recalls the...
23194    www.intouchweekly.com Tristan Thompson Feels "...
23195    www.billboard.com Kelly Clarkson Performs a Me...
Name: content, Length: 23196, dtype: object


In [17]:
# separating the data & label
X = news_dataset["content"]
Y = news_dataset['real']

In [18]:
print(X)
print(Y)

0        toofab.com Kandi Burruss Explodes Over Rape Ac...
1        www.today.com People's Choice Awards 2018: The...
2        www.etonline.com Sophia Bush Sends Sweet Birth...
3        www.dailymail.co.uk Colombian singer Maluma sp...
4        www.zerchoo.com Gossip Girl 10 Years Later: Ho...
                               ...                        
23191    www.express.co.uk Pippa Middleton wedding: In ...
23192    hollywoodlife.com Zayn Malik & Gigi Hadid’s Sh...
23193    www.justjared.com Jessica Chastain Recalls the...
23194    www.intouchweekly.com Tristan Thompson Feels "...
23195    www.billboard.com Kelly Clarkson Performs a Me...
Name: content, Length: 23196, dtype: object
0        1
1        1
2        1
3        1
4        1
        ..
23191    1
23192    0
23193    1
23194    0
23195    1
Name: real, Length: 23196, dtype: int64


In [19]:
port_stem = PorterStemmer()

In [20]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [21]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [22]:
print(news_dataset['content'])

0        toofab com kandi burruss explod rape accus rea...
1        www today com peopl choic award best red carpe...
2        www etonlin com sophia bush send sweet birthda...
3        www dailymail co uk colombian singer maluma sp...
4        www zerchoo com gossip girl year later upper e...
                               ...                        
23191    www express co uk pippa middleton wed case mis...
23192    hollywoodlif com zayn malik gigi hadid shock s...
23193    www justjar com jessica chastain recal moment ...
23194    www intouchweekli com tristan thompson feel du...
23195    www billboard com kelli clarkson perform medle...
Name: content, Length: 23196, dtype: object


In [23]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['real'].values

In [24]:
print(X)

['toofab com kandi burruss explod rape accus real housew atlanta reunion video'
 'www today com peopl choic award best red carpet look'
 'www etonlin com sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva'
 ...
 'www justjar com jessica chastain recal moment mother boyfriend slap kick genit'
 'www intouchweekli com tristan thompson feel dump khlo kardashian refus let move la home exclus'
 'www billboard com kelli clarkson perform medley kendrick lamar humbl hit billboard music award']


In [25]:
print(Y)

[1 1 1 ... 1 0 1]


In [26]:
Y.shape

(23196,)

In [27]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [28]:
print(X)

  (0, 13681)	0.21108934405813048
  (0, 12980)	0.29467349125048065
  (0, 10569)	0.2543303205934995
  (0, 10276)	0.23128301802698972
  (0, 10223)	0.30215839846863407
  (0, 6647)	0.3874939993343693
  (0, 5891)	0.2612462026749161
  (0, 4193)	0.3550616497873391
  (0, 2460)	0.05082186658865081
  (0, 1707)	0.3874939993343693
  (0, 651)	0.3073756565758506
  (0, 59)	0.2597097788538541
  (1, 14274)	0.10189043319535993
  (1, 12946)	0.36817044163275126
  (1, 10347)	0.37702288594702094
  (1, 9476)	0.23697414575567807
  (1, 7432)	0.33284454859410584
  (1, 2460)	0.08159478582438673
  (1, 2219)	0.42251489448694257
  (1, 1916)	0.3875096512688879
  (1, 1101)	0.33844294892026705
  (1, 729)	0.3083771474680274
  (2, 14274)	0.05440697837598742
  (2, 13118)	0.3043944712032482
  (2, 12381)	0.2330583052370701
  :	:
  (23194, 10393)	0.31860580642240854
  (23194, 8384)	0.28221547915154177
  (23194, 7230)	0.3087048153381826
  (23194, 6994)	0.26381957644897924
  (23194, 6797)	0.2878505190105094
  (23194, 6661)	0.1

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=2)


In [30]:
type(X_train)

scipy.sparse._csr.csr_matrix

In [31]:
model = LogisticRegression()

In [32]:
# training the Logistic Regression model with the training data
model.fit(X_train, Y_train)

LogisticRegression()

In [33]:
train_pred= model.predict(X_train)
test_pred= model.predict(X_test)
train_acc=accuracy_score(train_pred,Y_train)
test_acc=accuracy_score(test_pred,Y_test)
print("accuracy_score:\n",train_acc,test_acc)

accuracy_score:
 0.8839728389739168 0.8474137931034482
